# Plan real-world action using counterfactual example analysis and causal analysis

This notebook demonstrates the use of the Responsible AI Toolbox to make decisions from diabetes progression data. It walks through the API calls necessary to create a widget with causal inferencing insights, then guides a visual analysis of the data.

* [Launch Responsible AI Toolbox](#Launch-Responsible-AI-Toolbox)
    * [Train a Model](#Train-a-Model)
    * [Create Model and Data Insights](#Create-Model-and-Data-Insights)
* [Take Real-World Action](#Take-Real-World-Action)
    * [What-If Counterfactuals Analysis](#What-If-Counterfactuals-Analysis)
    * [Causal Analysis](#Causal-Analysis)

## Launch Responsible AI Toolbox

The following section examines the code necessary to create the dataset. It then generates insights using the `responsibleai` API that can be visually analyzed.

### Train a Model
*The following section can be skipped. It loads a dataset for illustrative purposes.*

In [1]:
import shap
import sklearn
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

First, load the diabetes dataset and specify the different types of features. Then, clean it and put it into a DataFrame with named columns.

In [2]:
data = sklearn.datasets.load_diabetes()
target_feature = 'y'
continuous_features = data.feature_names
data_df = pd.DataFrame(data.data, columns=data.feature_names)

After loading and cleaning the data, split the datapoints into training and test sets. Assemble separate datasets for the training and test data.

In [3]:
X_train, X_test, y_train, y_test = train_test_split(data_df, data.target, test_size=0.2, random_state=7)

train_data = X_train.copy()
test_data = X_test.copy()
train_data[target_feature] = y_train
test_data[target_feature] = y_test
X_train_drop = X_train.drop(['s1'], axis=1)
data.feature_names
X_train_drop

,age,sex,bmi,bp,s2,s3,s4,s5,s6
32,0.034443,0.050680,0.125287,0.028758,-0.012900,-0.102307,0.108111,0.000271,0.027917
403,-0.020045,-0.044642,0.097264,-0.005671,-0.023861,-0.021311,-0.002592,0.061686,0.040343
270,0.005383,0.050680,0.030440,0.083844,-0.047347,0.015505,-0.039493,0.008640,0.015491
57,-0.027310,-0.044642,-0.063330,-0.050428,-0.104340,0.052322,-0.076395,-0.056158,-0.067351
369,-0.009147,-0.044642,0.037984,-0.040099,-0.003819,-0.043401,0.015858,-0.005145,0.027917
...,...,...,...,...,...,...,...,...,...
211,0.092564,-0.044642,0.036907,0.021872,-0.016658,0.000779,-0.039493,-0.022512,-0.021788
67,0.041708,0.050680,-0.014828,-0.017147,0.008394,-0.013948,-0.001854,-0.011901,0.003064
25,-0.067268,0.050680,-0.012673,-0.040099,0.004636,-0.058127,0.034309,0.019199,-0.034215
196,-0.023677,-0.044642,-0.046085,-0.033214,0.036264,0.037595,-0.002592,-0.033249,0.011349


Train a nearest-neighbors classifier on the training data.

In [4]:
model = RandomForestRegressor(random_state=0)
model.fit(X_train_drop, y_train)

RandomForestRegressor(random_state=0)

### Create Model and Data Insights

In [5]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIInsights

To use Responsible AI Toolbox, initialize a RAIInsights object upon which different components can be loaded.

RAIInsights accepts the model, the train dataset, the test dataset, the target feature string, the task type string, and a list of strings of categorical feature names as its arguments.

In [6]:
from responsibleai.feature_metadata import FeatureMetadata

feature_metadata = FeatureMetadata(identity_feature_name='s1', dropped_features=['s1'])

rai_insights = RAIInsights(model, train_data, test_data, target_feature, 'regression',
                               categorical_features=[], feature_metadata=feature_metadata)

dropped_features are not in use currently.


Add the components of the toolbox that are focused on decision-making.

In [7]:
# Counterfactuals: accepts total number of counterfactuals to generate, the range that their label should fall under, 
# and a list of strings of categorical feature names
rai_insights.counterfactual.add(total_CFs=20, desired_range=[50, 120])
# Causal Inference: determines causation between features
rai_insights.causal.add(treatment_features=['bmi', 'bp', 's2'])
rai_insights.explainer.add(treatment_features=['bmi', 'bp', 's2'])

Once all the desired components have been loaded, compute insights on the test set.

In [8]:
rai_insights.compute()

100%|██████████| 89/89 [01:04<00:00,  1.38it/s]


Compose some cohorts which can be injected into the `ResponsibleAIDashboard`.

In [9]:
from raiwidgets.cohort import Cohort, CohortFilter, CohortFilterMethods

# Cohort on age and bmi features in the dataset
cohort_filter_age = CohortFilter(
    method=CohortFilterMethods.METHOD_LESS,
    arg=[40],
    column='age')
cohort_filter_bmi = CohortFilter(
    method=CohortFilterMethods.METHOD_GREATER,
    arg=[0],
    column='bmi')
    
user_cohort_age_and_bmi= Cohort(name='Cohort Age and BMI')
user_cohort_age_and_bmi.add_cohort_filter(cohort_filter_age)
user_cohort_age_and_bmi.add_cohort_filter(cohort_filter_bmi)

# Cohort on index
cohort_filter_index = CohortFilter(
    method=CohortFilterMethods.METHOD_LESS,
    arg=[20],
    column='Index')

user_cohort_index = Cohort(name='Cohort Index')
user_cohort_index.add_cohort_filter(cohort_filter_index)

# Cohort on predicted y values
cohort_filter_predicted_y = CohortFilter(
    method=CohortFilterMethods.METHOD_LESS,
    arg=[165.0],
    column='Predicted Y')

user_cohort_predicted_y = Cohort(name='Cohort Predicted Y')
user_cohort_predicted_y.add_cohort_filter(cohort_filter_predicted_y)

# Cohort on true y values
cohort_filter_true_y = CohortFilter(
    method=CohortFilterMethods.METHOD_GREATER,
    arg=[45.0],
    column='True Y')

user_cohort_true_y = Cohort(name='Cohort True Y')
user_cohort_true_y.add_cohort_filter(cohort_filter_true_y)

# Cohort on true y values
cohort_filter_regression_error = CohortFilter(
    method=CohortFilterMethods.METHOD_GREATER,
    arg=[20.0],
    column='Error')

user_cohort_regression_error = Cohort(name='Cohort Regression Error')
user_cohort_regression_error.add_cohort_filter(cohort_filter_regression_error)

cohort_list = [user_cohort_age_and_bmi,
               user_cohort_index,
               user_cohort_predicted_y,
               user_cohort_true_y,
               user_cohort_regression_error]

Finally, visualize and explore the model insights. Use the resulting widget or follow the link to view this in a new tab.

In [10]:
ResponsibleAIDashboard(rai_insights, cohort_list=cohort_list)

ResponsibleAI started at http://localhost:5000


The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- s1
Feature names must be in the same order as they were in fit.

Traceback (most recent call last):
  File "c:\users\gaugup\documents\github\responsible-ai-toolbox\raiwidgets\raiwidgets\responsibleai_dashboard_input.py", line 107, in on_predict
    self._analysis.model.predict(data), EXP_VIZ_ERR_MSG)
  File "c:\Users\gaugup\Anaconda3\envs\rai\lib\site-packages\sklearn\ensemble\_forest.py", line 961, in predict
    X = self._validate_X_predict(X)
  File "c:\Users\gaugup\Anaconda3\envs\rai\lib\site-packages\sklearn\ensemble\_forest.py", line 569, in _validate_X_predict
    X = self._validate_data(X, dtype=DTYPE, accept_sparse="csr", reset=False)
  File "c:\Users\gaugup\Anaconda3\envs\rai\lib\site-packages\sklearn\base.py", line 580, in _validate_data
    self._check_n_features(X, reset=reset)
  File "c:\Users\gaugup\Anaconda3\envs\rai\lib\si

X has 10 features, but RandomForestRegressor is expecting 9 features as input.


## Take Real-World Action

### What-If Counterfactuals Analysis

Let's imagine that the diabetes progression scores predicted by the model are used to determine medical insurance rates. If the score is greater than 120, there is a higher rate. Patient 43's model score of 268.08 results in this increased rate, and they want to know how they should change their health to get a lower rate prediction from the model (leading to lower insurance price).

The What-If counterfactuals component shows how slightly different feature values affect model predictions. This can be used to solve Patient 43's problem.

![What-If Counterfactuals component with datapoint 43 selected on the scatter plot with axes "Predicted Y" and "Index"](./img/regression-decision-making-1.png)

What can Patient 43 do to create the desired change? The top ranked features bar plot shows that `bmi` and `s5` are the best to perturb to bring the model score within 120.

![Top-ranked features (descending) for datapoint 43 to perturb to reduce model prediction below 120: bmi, s5, s4, s3, age, bp, sex, s1, s2, s6](./img/regression-decision-making-2.png)

Let's see how that can be achieved. Change `bmi` to -0.04 and `s5` to -0.042 and see what the result is.

![Counterfactual creation panel. BMI has been changed to -0.04 and s5 has been changed to -0.042](./img/regression-decision-making-3.png)

As we can see, the model's prediction has dropped to 131.22. Thus, Patient 43 should work on reducing their [body mass index and serum triglycerides level](https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset) to bring the model score under the insurance threshold.

![Counterfactual of datapoint 43 selected on the counterfactuals scatter plot with axes "Predicted Y" and "Index". Predicted Y is 115.4](./img/regression-decision-making-4.png)

Note that this result does not mean that reducing `bmi` and `s5` *causes* the diabetes progression score to go down. It simply decreases the model prediction. To investigate causal relationships, continue reading:

### Causal Analysis

Now suppose that a doctor wishes to know how to reduce the progression of diabetes in her patients. This can be explored in the Causal Inference component of the Responsible AI Toolbox.

In the "Aggregate causal effects" tab, it is possible to see how perturbing features causes lower disease progression. It appears that increasing s2 (LDL) by one unit, would increase diabetes progression by 606.89 units.

![Overall causal analysis table](./img/regression-decision-making-5.png)
![Causal analysis diagram](./img/regression-decision-making-6.png)

Let's revisit Patient 43. Instead of simply reducing the model score, they've decided to focus on actually improving their health to manage their diabetes better. In the "Individual causal what-if" tab, it shows that decreasing his/her bmi to 0.05 reduces diabetes progression from 242 to 237.982.

![individual causal analysis table](./img/regression-decision-making-7.png)

To put that into a formal intervention policy, switch to the "Treatment policy" tab. This view helps build policies for future interventions. You can identify what parts of your sample experience the largest responses to changes in causal features, or treatments, and construct rules to define which future populations should be targeted for particular interventions.

![treatment_policy](./img/regression-decision-making-8.png)

Is that change the best overall treatment for them? Let's investigate different policies. Going back to the "Treatment policy" tab, we see that going with the above intervention of s2 feature outperforms perturbing that  with a "always increase" intervention.

![image.png](./img/regression-decision-making-9.png)

Finally, you can see a list demonstrating which datapoints (patients) in the current data sample have the largest causal response to the selected treatment (s2 feature change), based on all features included in the estimated causal model.

![causal-table](./img/regression-decision-making-10.png)